# 数据集的划分

In [22]:
import pandas as pd
import os
import shutil
from tqdm import tqdm
import numpy as np
import librosa
import soundfile as sf
import scipy.io.wavfile as wavfile
import glob
from pathlib import Path

In [8]:
# 标注文件路径 (请确保这是包含 dataset_split 列的文件)
csv_path = r"X:\\数据集\\DeepShip\\data_preprocessing\\annotation\\DeepShip_No_Overlap_Metadata_add_Environmental_Noise.csv"
# 输入音频文件目录
SOURCE_ROOT = r"X:\\数据集\DeepShip\\data_preprocessing\\data_audio_rename_add_enviromental_noise_class"

# 输出目录 (程序会自动创建 train 和 test 文件夹)
OUTPUT_ROOT = r"X:\\数据集\\DeepShip\\data_preprocessing\\data_audio_rename_add_enviromental_noise_class_dataset_split"

In [7]:
def build_file_index(root_path):
    """
    不管文件是在 Cargo/0_1.wav 还是 Background/test/3_0/0000/4_0_1.wav
    只要在这个根目录下，都会被记录下来。
    返回字典: {'文件名': '完整绝对路径'}
    """
    print(f"🔍 正在扫描源目录建立索引: {root_path}")
    print("   (如果文件很多，这可能需要几秒钟...)")
    
    file_map = {}
    count = 0
    for root, dirs, files in os.walk(root_path):
        for file in files:
            if file.endswith('.wav'):
                # 记录: 文件名 -> 完整路径
                file_map[file] = os.path.join(root, file)
                count += 1
                
    print(f"✅ 索引建立完成！共找到 {count} 个音频文件。")
    return file_map

In [ ]:
def split_dataset_unified():
    # 读取 CSV
    if not os.path.exists(csv_path):
        print(f"❌ 错误: 找不到 CSV 文件 {csv_path}")
        return
    df = pd.read_csv(csv_path)
    
    # 只处理 train 和 test
    df_valid = df[df['dataset_split'].isin(['train', 'test'])]
    print(f"📋 待处理任务总数: {len(df_valid)}")

    # 建立文件索引 (解决路径深浅不一的问题)
    file_index = build_file_index(SOURCE_ROOT)
    
    if len(file_index) == 0:
        print("❌ 严重错误: 源目录下没有扫描到任何 .wav 文件！请检查路径。")
        return

    # 3.3 类别映射
    id_to_folder = {
        0: 'Cargo', 
        1: 'Passengership', 
        2: 'Tanker', 
        3: 'Tug', 
        4: 'Background'
    }

    success_count = 0
    missing_count = 0
    
    # 使用 tqdm 显示进度条
    for index, row in tqdm(df_valid.iterrows(), total=len(df_valid), desc="Splitting"):
        class_id = row['class_id']
        filename = row['new_filename'] # 例如 0_1.wav 或 4_0_1.wav
        split = row['dataset_split']   # train 或 test
        
        # 获取目标子文件夹名
        folder_name = id_to_folder.get(class_id)
        if not folder_name: continue

        # --- A. 查找源文件 (Source) ---
        # 直接查字典，不用管它在哪个子文件夹里
        src_path = file_index.get(filename)
        
        if not src_path:
            # 字典里没查到，说明文件真的不存在
            missing_count += 1
            if missing_count <= 5: # 只打印前5个错误，防止刷屏
                print(f"⚠️ 缺失: 找不到文件 {filename}")
            continue

        # --- B. 设定目标路径 (Destination) ---
        # 强制结构: Output / split / Category / filename
        dst_dir = os.path.join(OUTPUT_ROOT, split, folder_name)
        dst_path = os.path.join(dst_dir, filename)

        # --- C. 复制文件 ---
        try:
            os.makedirs(dst_dir, exist_ok=True)
            shutil.copy(src_path, dst_path)
            success_count += 1
        except Exception as e:
            print(f"❌ 复制失败 {filename}: {e}")

    # ================= 4. 结果统计 =================
    print("\n" + "="*30)
    print("处理完成！")
    print(f"✅ 成功复制: {success_count}")
    print(f"❌ 文件缺失: {missing_count}")
    print(f"📂 输出目录: {OUTPUT_ROOT}")
    
    # 打印目录结构确认
    print("\n生成的目录结构示例:")
    if success_count > 0:
        print(f"  {OUTPUT_ROOT}\\train\\Cargo\\0_1.wav")
        print(f"  {OUTPUT_ROOT}\\test\\Background\\4_0_1.wav")
        print("  (所有文件都直接位于类别文件夹下，无多余层级)")

In [10]:
# 运行主函数
split_dataset_unified()

📋 待处理任务总数: 28377
🔍 正在扫描源目录建立索引: X:\\数据集\DeepShip\\data_preprocessing\\data_audio_rename_add_enviromental_noise_class
   (如果文件很多，这可能需要几秒钟...)
✅ 索引建立完成！共找到 28377 个音频文件。


Splitting: 100%|██████████| 28377/28377 [03:30<00:00, 135.07it/s]


处理完成！
✅ 成功复制: 28377
❌ 文件缺失: 0
📂 输出目录: X:\\数据集\\DeepShip\\data_preprocessing\\data_audio_rename_add_enviromental_noise_class_dataset_split

生成的目录结构示例:
  X:\\数据集\\DeepShip\\data_preprocessing\\data_audio_rename_add_enviromental_noise_class_dataset_split\train\Cargo\0_1.wav
  X:\\数据集\\DeepShip\\data_preprocessing\\data_audio_rename_add_enviromental_noise_class_dataset_split\test\Background\4_0_1.wav
  (所有文件都直接位于类别文件夹下，无多余层级)


# 因为噪声类已经切割过了，所以不做切割处理，等其它类做了切割后再一起做预处理

In [14]:
INPUT_ROOT = Path(r"X:\\数据集\\DeepShip\\data_preprocessing\\data_audio_rename_add_enviromental_noise_class_dataset_split")
OUTPUT_ROOT = Path(r"X:\\数据集\\DeepShip\\data_preprocessing\\data_audio_rename_add_enviromental_noise_class_dataset_split_preprocessed")

# 音频参数
FRAME_DURATION = 3.0   # 切片时长 3秒
OVERLAP_RATE = 0.0     # 0重叠
TARGET_SR = 32000      # 目标采样率
TARGET_RMS = 0.05      # 目标 RMS 值

In [ ]:
def remove_mean(frame: np.ndarray) -> np.ndarray:
    """去中心化: 移除直流分量"""
    return frame - np.mean(frame)

def rms_normalize(frame: np.ndarray, target_rms: float = 0.05) -> np.ndarray:
    """RMS 归一化"""
    rms = np.sqrt(np.mean(frame ** 2) + 1e-12)
    if rms < 1e-8:
        # 如果是静音片段，直接返回全0
        return np.zeros_like(frame)
    scale = target_rms / rms
    return frame * scale

def process_and_save(audio_data, save_path, sr):
    """统一的后处理流水线：去中心化 -> 归一化 -> 保存"""
    # 去中心化
    audio_data = remove_mean(audio_data)
    
    # 归一化
    audio_data = rms_normalize(audio_data, target_rms=TARGET_RMS)
    
    # 保存 (float32)
    wavfile.write(save_path, sr, audio_data.astype(np.float32))

In [17]:
def preprocess_dataset_smart_resample():
    if not INPUT_ROOT.exists():
        print(f"❌ 错误: 输入目录不存在 {INPUT_ROOT}")
        return

    # 遍历 train 和 test
    for split in ["train", "test"]:
        split_in_dir = INPUT_ROOT / split
        split_out_dir = OUTPUT_ROOT / split
        
        if not split_in_dir.exists():
            continue

        print(f"\n🚀 正在处理数据集划分: {split} ...")

        # 获取当前划分下的所有类别文件夹
        categories = [d.name for d in split_in_dir.iterdir() if d.is_dir()]
        
        for category in categories:
            in_cat_dir = split_in_dir / category
            out_cat_dir = split_out_dir / category
            out_cat_dir.mkdir(parents=True, exist_ok=True)
            
            wav_paths = sorted(list(in_cat_dir.glob("*.wav")))
            if not wav_paths:
                continue
            
            print(f"📂 正在处理类别: {category} ({len(wav_paths)} 个文件)")

            # 判断是否为背景噪声类
            is_noise_class = (category == "Background")

            for wav_path in tqdm(wav_paths, desc=f"     Processing {category}", leave=False):
                try:
                    # 先使用 sr=None 加载，获取原始采样率
                    audio, sr = librosa.load(wav_path, sr=None)
                    
                    # 只有当采样率不匹配时，才进行重采样
                    if sr != TARGET_SR:
                        print(f"重采样: {wav_path.name} ({sr}Hz -> {TARGET_SR}Hz)")
                        audio = librosa.resample(audio, orig_sr=sr, target_sr=TARGET_SR)
                        sr = TARGET_SR
                    
                    file_stem = wav_path.stem 
                    
                    # 场景 A: 背景噪声类 (不切割)
                    if is_noise_class:
                        target_len = int(FRAME_DURATION * sr)
                        if len(audio) < target_len:
                            audio = np.pad(audio, (0, target_len - len(audio)), mode='constant')
                        elif len(audio) > target_len:
                            audio = audio[:target_len]
                        
                        save_name = f"{file_stem}.wav"
                        process_and_save(audio, out_cat_dir / save_name, sr)

                    # 场景 B: 船只类 (切割，重命名为 X_X_1.wav)
                    else:
                        frame_length = int(FRAME_DURATION * sr)
                        hop_length = int(frame_length * (1.0 - OVERLAP_RATE))
                        
                        if len(audio) < frame_length:
                            audio = np.pad(audio, (0, frame_length - len(audio)), mode='constant')
                        
                        frames = librosa.util.frame(audio, frame_length=frame_length, hop_length=hop_length).T
                        
                        # start=1: 索引从1开始
                        for i, frame in enumerate(frames, start=1):
                            save_name = f"{file_stem}_{i}.wav"
                            process_and_save(frame, out_cat_dir / save_name, sr)
                            
                except Exception as e:
                    print(f"❌ 处理出错 {wav_path.name}: {e}")

    print("\n✅ 所有文件预处理完成！")
    print(f"输出目录: {OUTPUT_ROOT}")

In [18]:
# 运行处理
preprocess_dataset_smart_resample()


🚀 正在处理数据集划分: train ...
📂 正在处理类别: Background (19635 个文件)


📂 正在处理类别: Cargo (78 个文件)


📂 正在处理类别: Passengership (120 个文件)


📂 正在处理类别: Tanker (158 个文件)


📂 正在处理类别: Tug (42 个文件)


...

📂 正在处理类别: Cargo (31 个文件)


📂 正在处理类别: Passengership (71 个文件)


📂 正在处理类别: Tanker (82 个文件)


📂 正在处理类别: Tug (27 个文件)



✅ 所有文件预处理完成！
输出目录: X:\数据集\DeepShip\data_preprocessing\data_audio_rename_add_enviromental_noise_class_dataset_split_preprocessed


# 添加高斯白噪声

In [19]:
SOURCE_ROOT = Path(r"X:\数据集\DeepShip\data_preprocessing\data_audio_rename_add_enviromental_noise_class_dataset_split_preprocessed")

ANNOTATION_CSV = Path(r"X:\数据集\DeepShip\data_preprocessing\annotation\DeepShip_No_Overlap_add_Environmental_Noise_Segmented_Metadata.csv")

DEST_ROOT = Path(r"X:\数据集\DeepShip\data_preprocessing\data_audio_rename_add_enviromental_noise_class_frame_and_window_3s_0%_16kHz_train_val_test_RMS_noisy_dataset")

In [20]:
TRAIN_RANGE = (-12, 6)
EVAL_SNRS = np.array([-12, -9, -6, -3, 0, 3, 6], dtype=np.int32)

ID_TO_FOLDER = {
    0: 'Cargo',
    1: 'Passengership',
    2: 'Tanker',
    3: 'Tug',
    4: 'Background'
}

In [ ]:
def add_noise_with_snr(signal, target_snr_db, rng):
    """
    根据目标 SNR 添加高斯白噪声
    """
    # 计算信号功率 (避免除以0，加极小值)
    signal_power = np.mean(signal ** 2) + 1e-12
    
    # 将 dB 转为线性比例: SNR = P_signal / P_noise
    snr_linear = 10 ** (target_snr_db / 10.0)
    
    # 计算需要的噪声功率
    noise_power = signal_power / snr_linear
    
    # 生成高斯白噪声 (均值0, 标准差 sqrt(power))
    noise = rng.normal(0.0, np.sqrt(noise_power), size=signal.shape)
    
    return signal + noise

def get_wav_path(root, split, class_id, filename):
    """构建带类别子文件夹的路径"""
    folder_name = ID_TO_FOLDER.get(class_id)
    if not folder_name:
        raise ValueError(f"未知的 class_id: {class_id}")
    return root / split / folder_name / filename, folder_name

# ================= 4. 主处理逻辑 =================

def generate_noisy_dataset():
    if not ANNOTATION_CSV.exists():
        print(f"❌ 错误: 找不到 CSV 文件 {ANNOTATION_CSV}")
        return

    df = pd.read_csv(ANNOTATION_CSV)
    print(f"📄 加载标注文件，共 {len(df)} 条数据")
    
    # 初始化随机生成器
    train_rng = np.random.default_rng(42)
    
    # 确保输出目录存在
    DEST_ROOT.mkdir(parents=True, exist_ok=True)

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Generating Noise"):
        # 1. 解析信息 (根据您之前的CSV列名)
        # 注意: 这里假设您的CSV列名为 'dataset_split', 'class_id', 'new_filename'
        # 如果您的列名不同，请在此处修改，例如 row['split']
        split = row.get("dataset_split", row.get("split")) 
        class_id = row.get("class_id", 0) # 默认为0或报错
        fname = row.get("new_filename", row.get("segmented_filename"))
        
        # 2. 读取原始音频
        try:
            wav_path, folder_name = get_wav_path(SOURCE_ROOT, split, class_id, fname)
            if not wav_path.exists():
                # 尝试容错: 有些时候 filename 可能不带后缀
                if not str(wav_path).endswith('.wav'):
                    wav_path = wav_path.with_suffix('.wav')
                
                if not wav_path.exists():
                    # print(f"⚠️ 缺失文件: {wav_path}")
                    continue

            audio, sr = sf.read(wav_path)
            audio = audio.astype(np.float32)
        except Exception as e:
            print(f"❌ 读取错误 {fname}: {e}")
            continue

        # ==========================================
        # 逻辑 A: 训练集 (随机 SNR，保持目录结构)
        # ==========================================
        if split == "train":
            # 随机选择一个 SNR
            snr = train_rng.integers(TRAIN_RANGE[0], TRAIN_RANGE[1] + 1)
            
            # 使用随机种子生成噪声
            seed = train_rng.integers(0, 2**32 - 1)
            noisy_audio = add_noise_with_snr(audio, snr, np.random.default_rng(seed))
            
            # 保存路径: DEST / train / Category / file.wav
            out_dir = DEST_ROOT / "train" / folder_name
            out_dir.mkdir(parents=True, exist_ok=True)
            
            sf.write(out_dir / fname, noisy_audio, sr, subtype="FLOAT")

        # ==========================================
        # 逻辑 B: 测试集/验证集 (固定 SNR 组，分文件夹存放)
        # ==========================================
        else: # test
            for snr in EVAL_SNRS:
                # 确定性种子 (保证复现性: 同一个文件在同一个SNR下生成的噪声永远一样)
                seed_input = f"{fname}-{snr}"
                seed = np.uint32(abs(hash(seed_input)) & 0xFFFFFFFF)
                
                noisy_audio = add_noise_with_snr(audio, snr, np.random.default_rng(seed))
                
                # 保存路径: DEST / test / SNR_XdB / Category / file.wav
                snr_folder_name = f"SNR_{snr:+d}dB"
                out_dir = DEST_ROOT / split / snr_folder_name / folder_name
                out_dir.mkdir(parents=True, exist_ok=True)
                
                sf.write(out_dir / fname, noisy_audio, sr, subtype="FLOAT")

    print("\n✅ 处理完成！")
    print(f"输出目录: {DEST_ROOT}")

In [24]:
# 运行
generate_noisy_dataset()

📄 加载标注文件，共 84236 条数据


Generating Noise: 100%|██████████| 84236/84236 [42:58<00:00, 32.67it/s]


✅ 处理完成！
输出目录: X:\数据集\DeepShip\data_preprocessing\data_audio_rename_add_enviromental_noise_class_frame_and_window_3s_0%_16kHz_train_val_test_RMS_noisy_dataset
